In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

Source_Data='/Volumes/workspace/data/employee_etl/Source_Data'
Temp='/Volumes/workspace/data/employee_etl/tmp'
Output='/Volumes/workspace/data/employee_etl/Output'

In [0]:
df_master=spark.read.csv(Source_Data+'/employee_master.csv',header=True,inferSchema=True)
df_master.display()

Emp_Id,Emp_Name,Emp_Manager,Emp_Lan,Emp_Age,Emp_Gender,Emp_Exp
E001,John Doe,Michael Brown,English,34,Male,6
E002,Jane Smith,Michael Brown,English,29,Female,3
E003,Akira Tanaka,Satoshi Ito,Japanese,41,Male,12
E004,Maria Garcia,Ana Lopez,Spanish,37,Female,9
E005,Chen Wei,Li Zhang,Chinese,30,Male,5


In [0]:
df_jira=spark.read.parquet(Temp+'/Jira_LRF.parquet',header=True,inferSchema=True)
df_jira.display()

Emp_Id,Jira_Points,Percentage
E001,128,64.0
E002,95,47.5
E003,152,76.0
E004,86,43.0
E005,110,55.0


In [0]:
df_rto=spark.read.parquet(Temp+'/RTO_LRF.parquet',header=True,inferSchema=True)
df_rto.display()

Emp_Id,RTO
E003,Complaint


In [0]:
df=df_master.join(df_jira,df_master.Emp_Id==df_jira.Emp_Id,'inner')
df=df.select(df_master.Emp_Id.alias('Emp_Id'), 'Emp_Name','Emp_Manager','Emp_Lan','Emp_Age','Emp_Gender','Emp_Exp','Jira_Points','Percentage')
df.display()

Emp_Id,Emp_Name,Emp_Manager,Emp_Lan,Emp_Age,Emp_Gender,Emp_Exp,Jira_Points,Percentage
E001,John Doe,Michael Brown,English,34,Male,6,128,64.0
E002,Jane Smith,Michael Brown,English,29,Female,3,95,47.5
E003,Akira Tanaka,Satoshi Ito,Japanese,41,Male,12,152,76.0
E004,Maria Garcia,Ana Lopez,Spanish,37,Female,9,86,43.0
E005,Chen Wei,Li Zhang,Chinese,30,Male,5,110,55.0


In [0]:
df=df.join(df_rto,df.Emp_Id==df_rto.Emp_Id,'left')
df = df.fillna('Non-Complaint', subset=['RTO'])
df=df.drop(df_rto.Emp_Id)
df.display()

Emp_Id,Emp_Name,Emp_Manager,Emp_Lan,Emp_Age,Emp_Gender,Emp_Exp,Jira_Points,Percentage,RTO
E001,John Doe,Michael Brown,English,34,Male,6,128,64.0,Non-Complaint
E002,Jane Smith,Michael Brown,English,29,Female,3,95,47.5,Non-Complaint
E003,Akira Tanaka,Satoshi Ito,Japanese,41,Male,12,152,76.0,Complaint
E004,Maria Garcia,Ana Lopez,Spanish,37,Female,9,86,43.0,Non-Complaint
E005,Chen Wei,Li Zhang,Chinese,30,Male,5,110,55.0,Non-Complaint


In [0]:
df=df.withColumn(
    'Flag',
    when((col("RTO") == 'Complaint') & (col("Percentage") >= 50), 1).otherwise(0)
)
df.display()

Emp_Id,Emp_Name,Emp_Manager,Emp_Lan,Emp_Age,Emp_Gender,Emp_Exp,Jira_Points,Percentage,RTO,Flag
E001,John Doe,Michael Brown,English,34,Male,6,128,64.0,Non-Complaint,0
E002,Jane Smith,Michael Brown,English,29,Female,3,95,47.5,Non-Complaint,0
E003,Akira Tanaka,Satoshi Ito,Japanese,41,Male,12,152,76.0,Complaint,1
E004,Maria Garcia,Ana Lopez,Spanish,37,Female,9,86,43.0,Non-Complaint,0
E005,Chen Wei,Li Zhang,Chinese,30,Male,5,110,55.0,Non-Complaint,0


In [0]:
grade_df=df.filter(col("Flag")==1).drop("Flag")
negative_df=df.filter(col("Flag")==0).drop("Flag")



In [0]:
del df,df_master,df_jira,df_rto


In [0]:
grade_df.display()

Emp_Id,Emp_Name,Emp_Manager,Emp_Lan,Emp_Age,Emp_Gender,Emp_Exp,Jira_Points,Percentage,RTO
E003,Akira Tanaka,Satoshi Ito,Japanese,41,Male,12,152,76.0,Complaint


In [0]:
negative_df.display()

Emp_Id,Emp_Name,Emp_Manager,Emp_Lan,Emp_Age,Emp_Gender,Emp_Exp,Jira_Points,Percentage,RTO
E001,John Doe,Michael Brown,English,34,Male,6,128,64.0,Non-Complaint
E002,Jane Smith,Michael Brown,English,29,Female,3,95,47.5,Non-Complaint
E004,Maria Garcia,Ana Lopez,Spanish,37,Female,9,86,43.0,Non-Complaint
E005,Chen Wei,Li Zhang,Chinese,30,Male,5,110,55.0,Non-Complaint


In [0]:
grade_df.write.mode("overwrite").parquet(Output+'/Grade.parquet')



In [0]:
negative_df.write.mode("overwrite").parquet(Temp+'/Negative_LRF.parquet')